In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('Review.csv')
print(df.columns)

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')


In [ ]:
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
df.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')

In [2]:
# Fill NA values in 'Review Text' with an empty string
df['Review Text'] = df['Review Text'].fillna('')

# Create the 'Sentiment' column based on the 'Rating' column
# For example, ratings of 4 or higher are considered positive (1) and 2 or lower are considered negative (0)
df['Sentiment'] = df['Rating'].apply(lambda rating: 1 if rating >= 4 else 0)


In [3]:
sentiment_counts = df['Sentiment'].value_counts()
print(sentiment_counts)

Sentiment
1    18208
0     5278
Name: count, dtype: int64


In [4]:
pip install deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00


In [ ]:
from imblearn.over_sampling import ADASYN
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(df['Review Text'])

X = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
y = df['Sentiment']

adasyn = ADASYN()
X_resampled, y_resampled = adasyn.fit_resample(X, y)

df_resampled = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='Sentiment')], axis=1)

print(df_resampled['Sentiment'].value_counts())

In [ ]:
from imblearn.over_sampling import ADASYN
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

vectorizer = TfidfVectorizer(min_df=5)
X_tfidf = vectorizer.fit_transform(df['Review Text'])

X = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
y = df['Sentiment']

adasyn = ADASYN()
X_resampled, y_resampled = adasyn.fit_resample(X, y)

df_resampled = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='Sentiment')], axis=1)

print(df_resampled['Sentiment'].value_counts())


In [ ]:
df_resampled.to_csv('output1.csv', index=False)

In [5]:
from deep_translator import GoogleTranslator
import pandas as pd

BATCH_SIZE = 20

def back_translate(text_data):
    translator = GoogleTranslator(source='auto', target='fr')
    back_translator = GoogleTranslator(source='fr', target='en')

    back_translated_texts = []
    for text in text_data:
        translated_text = translator.translate(text)
        back_translated_text = back_translator.translate(translated_text)
        back_translated_texts.append(back_translated_text)

    return back_translated_texts

X = df['Review Text']
y = df['Sentiment']

minority_class_data = X[y == 0]

text_list = minority_class_data.tolist()
text_batches = [text_list[i:i+BATCH_SIZE] for i in range(0, len(text_list), BATCH_SIZE)]
augmented_data = []
for batch in text_batches:
    back_translated_batch = back_translate(batch)
    augmented_data.extend(back_translated_batch)

augmented_df = pd.DataFrame({'Review': augmented_data, 'Sentiment': 0})
df_augmented = pd.concat([df, augmented_df], ignore_index=True)

print(df_augmented['Sentiment'].value_counts())


Sentiment
1    18208
0    10556
Name: count, dtype: int64


In [6]:
df_augmented.to_csv('output2.csv', index=False)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense, BatchNormalization
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import confusion_matrix, classification_report

df = pd.read_csv('output2.csv')
df['Review Text'] = df['Review Text'].fillna('')
df['Sentiment'] = df['Rating'].apply(lambda rating: 1 if rating >= 4 else 0)

In [10]:
from tensorflow.keras.layers import Bidirectional, LSTM, Embedding, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import LeakyReLU
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Review Text'])
sequences = tokenizer.texts_to_sequences(df['Review Text'])
max_sequence_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)
labels = np.array(df['Sentiment'])
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, train_size=0.7, random_state=42)

In [13]:
#RNN BiLSTM
from tensorflow.keras.layers import Bidirectional, BatchNormalization

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=200, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(units=128, dropout=0.25, recurrent_dropout=0.25, return_sequences=True)))
model.add(Bidirectional(LSTM(units=64, dropout=0.25, recurrent_dropout=0.25)))
#model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=1)

model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=128,callbacks=[early_stopping])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

# Predict the labels for the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary output

# Print the confusion matrix and classification report
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

print('Confusion Matrix:\n', cm)
print('Classification Report:\n', cr)

Epoch 1/10
126/126 [==============================] - 535s 4s/step - loss: 0.4014 - accuracy: 0.8326 - val_loss: 0.3399 - val_accuracy: 0.8617
Epoch 2/10
126/126 [==============================] - 444s 4s/step - loss: 0.2772 - accuracy: 0.8958 - val_loss: 0.3388 - val_accuracy: 0.8676
Epoch 3/10
126/126 [==============================] - 431s 3s/step - loss: 0.2316 - accuracy: 0.9159 - val_loss: 0.3414 - val_accuracy: 0.8773
Epoch 4/10
270/270 [==============================] - 44s 162ms/step - loss: 0.4078 - accuracy: 0.8645
Test loss: 0.4078
Test accuracy: 0.8645
270/270 [==============================] - 44s 160ms/step
Confusion Matrix:
 [[2540  657]
 [ 512 4921]]
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.79      0.81      3197
           1       0.88      0.91      0.89      5433

    accuracy                           0.86      8630
   macro avg       0.86      0.85      0.85      8630
weighted avg       0.86     

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.data import Dataset
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
inputs = tokenizer(df['Review Text'].tolist(), truncation=True, padding=True, return_tensors='tf')

# Convert the labels to a TensorFlow tensor
labels = tf.convert_to_tensor(df['Sentiment'].values)

# Create a TensorFlow dataset
dataset = Dataset.from_tensor_slices((dict(inputs), labels))

# Split the dataset into training and testing sets
dataset = dataset.shuffle(len(df))
train_size = int(0.7 * len(df))
test_size = len(df) - train_size
train_dataset, test_dataset = dataset.take(train_size), dataset.skip(train_size)

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

loss_fn = BinaryCrossentropy(from_logits=True)
optimizer = Adam(learning_rate=3e-5)

# Compile the model (using the defined optimizer object)
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

model.fit(train_dataset.shuffle(1000).batch(16), epochs=1, batch_size=16, validation_data=test_dataset.shuffle(1000).batch(16))

# Evaluate the model on the test set
results = model.evaluate(test_dataset.batch(16), batch_size=16)
print(f'Test loss: {results[0]:.4f}')
print(f'Test accuracy: {results[1]:.4f}')


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
  29/1259 [..............................] - ETA: 10:09:38 - loss: 0.7616 - accuracy: 0.4569

In [ ]:
# Predict the labels for the test set
predictions = model.predict(test_dataset.batch(16))
y_pred = tf.nn.sigmoid(predictions.logits)  # Convert logits to probabilities
y_pred = (y_pred > 0.5)  # Convert probabilities to binary output

# Convert the TensorFlow tensors to numpy arrays for evaluation
y_test = df['Sentiment'].values
y_pred = y_pred.numpy().flatten()

# Print the confusion matrix and classification report
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

print('Confusion Matrix:\n', cm)
print('Classification Report:\n', cr)

In [9]:
#CNN OOV
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

df['Review Text'] = df['Review Text'].astype(str)
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['Review Text'])
sequences = tokenizer.texts_to_sequences(df['Review Text'])
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')
df['Sentiment'] = df['Rating'].apply(lambda x: 1 if x >= 4 else 0)
labels = df['Sentiment'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

model = Sequential([
    Embedding(5000, 100, input_length=100),
    Conv1D(32, 5, activation='relu'),
    Dropout(0.5),  # Add dropout after the Conv1D layer
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=1)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=128, callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

y_pred = (model.predict(X_test) > 0.5).astype("int32")

print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n', cm)


Epoch 1/10
180/180 [==============================] - 12s 60ms/step - loss: 0.4521 - accuracy: 0.8140 - val_loss: 0.4148 - val_accuracy: 0.8472
Epoch 2/10
180/180 [==============================] - 9s 50ms/step - loss: 0.3100 - accuracy: 0.8771 - val_loss: 0.3620 - val_accuracy: 0.8710
Epoch 3/10
180/180 [==============================] - 1s 4ms/step - loss: 0.3502 - accuracy: 0.8708
Test accuracy: 87.08%
180/180 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

           0       0.83      0.81      0.82      2100
           1       0.89      0.91      0.90      3653

    accuracy                           0.87      5753
   macro avg       0.86      0.86      0.86      5753
weighted avg       0.87      0.87      0.87      5753

Confusion Matrix:
 [[1702  398]
 [ 345 3308]]
